In [46]:
!which python

/home/jenna/dissertation_chapters/cmems_ch3_project/.venv/bin/python


In [25]:
import folium
from folium import Marker
from folium.plugins import Draw
from shapely.geometry import Point, Polygon
from shapely.ops import transform
import pyproj
import geopandas as gpd

In [26]:
# Stranding point
lat, lon = 44.64, -124.0535

In [36]:
# Projection to work in meters to allow for N/S/E/W shifting
proj_wgs84 = pyproj.CRS("EPSG:4326")
proj_meters = pyproj.CRS("EPSG:3857")  # Mercator

# Create a transformer
project = pyproj.Transformer.from_crs(proj_wgs84, proj_meters, always_xy=True).transform
reproject = pyproj.Transformer.from_crs(proj_meters, proj_wgs84, always_xy=True).transform

# Convert to meters
point_m = transform(project, Point(lon, lat))

# Define distances in meters (60km N/S, 100km W "offshore")
buffer_north = 60_000
buffer_south = 120_000
buffer_west = 200_000
buffer_east = 3_000  # just a bit inland to ensure overlap

# Create rectangle in projected space
minx = point_m.x - buffer_west
maxx = point_m.x + buffer_east
miny = point_m.y - buffer_south
maxy = point_m.y + buffer_north
rectangle_m = Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny)])

# Reproject back to lat/lon
rectangle_wgs84 = transform(reproject, rectangle_m)

# Get bounds: (minx, miny, maxx, maxy) → (west, south, east, north)
west, south, east, north = rectangle_wgs84.bounds

print(f"Bounding box coordinates for CMEMS query:")
print(f"North: {north}")
print(f"South: {south}")
print(f"East:  {east}")
print(f"West:  {west}")

Bounding box coordinates for CMEMS query:
North: 45.022242486335735
South: 43.86791088300122
East:  -124.0265505414764
West:  -125.85013056823904


In [37]:
# GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=[rectangle_wgs84], crs="EPSG:4326")

# Map with ESRI satellite view
m = folium.Map(
    location=[lat, lon],
    zoom_start=6,
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Tiles © Esri"
)

# Add marker and rectangle
folium.Marker([lat, lon], popup="Stranding", icon=folium.Icon(color="purple", icon="dot")).add_to(m)
folium.GeoJson(gdf).add_to(m)

m